# **Sentiment Analysis of IMDB reviews**

import CSV

In [53]:
import pandas as pd

df = pd.read_csv("/content/movie_Sentiment_analysis.csv")

df.head(5)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [54]:
df.isnull().sum()

,0
text,0
label,0


In [55]:
df.dtypes

,0
text,object
label,int64


Check imbalance

In [56]:
df["label"].value_counts()

,count
label,
0,20019
1,19981


In [57]:
pip install nltk


In [58]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd

Reducing number of rows as the model was taking alot of time to process

In [59]:
n_samples = 5000  # Total number of rows you want to sample
n_classes = df['label'].nunique()

# Calculate how many samples per label
n_samples_per_label = n_samples // n_classes

# Perform stratified sampling: sample proportionally from each class
df= df.groupby('label').apply(lambda x: x.sample(n=n_samples_per_label, random_state=42)).reset_index(drop=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)



<ipython-input-59-21ad6068351f>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df= df.groupby('label').apply(lambda x: x.sample(n=n_samples_per_label, random_state=42)).reset_index(drop=True)


In [60]:
df["label"].value_counts()

,count
label,
0,2500
1,2500


pre processing of the data

In [61]:
# Download necessary NLTK data
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords

# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    # 1. Lowercase the text
    text = text.lower()

    # 2. Tokenize the text
    words = word_tokenize(text)

    # 3. Remove stopwords and perform stemming
    processed_words = [stemmer.stem(word) for word in words if word.isalpha() and word not in stop_words]

    # Return processed words as a single string (or you can keep it as a list if needed)
    return ' '.join(processed_words)

# Apply preprocessing to the 'text' column
df['processed_text'] = df['text'].apply(preprocess_text)

# Display the DataFrame
df.head(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,processed_text
0,"Okay this is stupid,they say their not making ...",0,okay stupid say make anoth nightmar film last ...
1,I saw this at the 2004 Stony Brook Film Festiv...,1,saw stoni brook film festiv ny warmli receiv f...
2,As I am always looking for something new and u...,1,alway look someth new uniqu watch film onlin t...
3,"Before seeing this movie, I was expecting a fi...",0,see movi expect fiction drama base loos idea b...
4,Why was this movie ever made?They have tarnish...,0,movi ever made tarnish origin caddyshack abl w...


In [62]:
df=df[["processed_text", "label"]]
df.head(5)

,processed_text,label
0,okay stupid say make anoth nightmar film last ...,0
1,saw stoni brook film festiv ny warmli receiv f...,1
2,alway look someth new uniqu watch film onlin t...,1
3,see movi expect fiction drama base loos idea b...,0
4,movi ever made tarnish origin caddyshack abl w...,0


change text data into vector form

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents
X = tfidf_vectorizer.fit_transform(df['processed_text'])
y=df["label"]


Split Data into train test data

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Random Forest Classifier to predict the label class

In [65]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [66]:
y_pred_label = rf.predict(X_test)


Evaluation metrics

In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

rf_accuracy = accuracy_score(y_test, y_pred_label)
print("Accuracy:", rf_accuracy)

# Model precision
rf_precision= precision_score(y_test, y_pred_label)
print("Precision:", rf_precision)

# Model recall
rf_recall= recall_score(y_test, y_pred_label)
print("Recall:", rf_recall)

# Model f1_score
rf_f1score= f1_score(y_test, y_pred_label)
print("F1_score:", rf_f1score)

Accuracy: 0.828
Precision: 0.8682008368200836
Recall: 0.7919847328244275
F1_score: 0.8283433133732535


Manual review Test

In [70]:
# Now, to predict the label for a single new text input
text = input("Enter the text for label prediction: ")

text_transformed = tfidf_vectorizer.transform([text])

# Use the trained model to predict the label
predicted_label = rf.predict(text_transformed)

# Map the prediction to 'positive' or 'negative'
if predicted_label == 1:
    result = "positive"
else:
    result = "negative"

# Print the result
print(f"Predicted label for the input text: {result}")


Enter the text for label prediction: this Analyser works well
Predicted label for the input text: positive
